linear regress this thing

In [5]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

import env
import acquire

In [6]:
df = acquire.get_telco_churn_data()

In [7]:
df.head()

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,online_security,...,device_protection,tech_support,streaming_tv,streaming_movies,contract_type_id,paperless_billing,payment_type_id,monthly_charges,total_charges,churn
0,0002-ORFBO,Female,0,Yes,Yes,9,Yes,No,1,No,...,No,Yes,Yes,No,2,Yes,2,65.6,593.3,No
1,0003-MKNFE,Male,0,No,No,9,Yes,Yes,1,No,...,No,No,No,Yes,1,No,2,59.9,542.4,No
2,0004-TLHLJ,Male,0,No,No,4,Yes,No,2,No,...,Yes,No,No,No,1,Yes,1,73.9,280.85,Yes
3,0011-IGKFF,Male,1,Yes,No,13,Yes,No,2,No,...,Yes,No,Yes,Yes,1,Yes,1,98.0,1237.85,Yes
4,0013-EXCHZ,Female,1,Yes,No,3,Yes,No,2,No,...,No,Yes,Yes,No,1,Yes,2,83.9,267.4,Yes


In [8]:
df['churn'] = df['churn'].replace('Yes', 1)
df['churn'] = df['churn'].replace('No', 0)

In [9]:
df.replace(r'^\s*$', np.nan, regex=True, inplace=True)
df["total_charges"] = df["total_charges"].astype('float')
df = df.dropna()

In [10]:
X = df[['senior_citizen', 'tenure', 'internet_service_type_id', 'contract_type_id', 'payment_type_id', 'monthly_charges', 'total_charges']]
y = df[['churn']]

In [11]:
X_train.head()

,gender,senior_citizen,streaming_tv,streaming_movies,has_phone_service,family,security_package,DSL,Fiber Optic,None,Month-to-Month,One Year,Two Year,tenure_scaled,monthly_charges_scaled,total_charges_scaled
5483,1,0,0,0,True,True,False,0.0,1.0,0.0,1.0,0.0,0.0,0.197183,0.500249,0.125756
5436,1,0,0,1,True,True,True,1.0,0.0,0.0,0.0,0.0,1.0,0.985915,0.614848,0.642349
3572,0,0,0,0,True,True,True,1.0,0.0,0.0,1.0,0.0,0.0,0.436620,0.428500,0.212999
2100,1,0,0,0,True,True,False,0.0,0.0,1.0,1.0,0.0,0.0,0.253521,0.015944,0.041397
7019,1,0,1,1,True,True,True,0.0,1.0,0.0,0.0,0.0,1.0,0.985915,0.879920,0.849694


In [12]:
logit = LogisticRegression(C=1, class_weight={1:2}, random_state = 123, solver='saga')

In [13]:
logit.fit(X_train, y_train)

LogisticRegression(C=1, class_weight={1: 2}, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=123, solver='saga', tol=0.0001, verbose=0,
                   warm_start=False)

In [14]:
print('Coefficient: \n', logit.coef_)
print('Intercept: \n', logit.intercept_)

Coefficient: 
 [[ 0.03880402  0.35699757  0.30894732  0.33251987 -0.38107305 -0.08744199
  -0.39846868 -0.03394136  1.11980217 -1.08096645  0.87689401 -0.03904851
  -0.83295114 -3.10204531 -0.36759037  1.38249416]]
Intercept: 
 [-0.00900733]


In [15]:
y_pred = logit.predict(X_train)

In [16]:
y_pred_proba = logit.predict_proba(X_train)

In [17]:
print('Accuracy of Logistic Regression classifier on training set: {:.2f}'
     .format(logit.score(X_train, y_train)))

Accuracy of Logistic Regression classifier on training set: 0.77


In [18]:
print(confusion_matrix(y_train, y_pred))

[[3234  892]
 [ 403 1096]]


In [19]:
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       0.89      0.78      0.83      4126
           1       0.55      0.73      0.63      1499

    accuracy                           0.77      5625
   macro avg       0.72      0.76      0.73      5625
weighted avg       0.80      0.77      0.78      5625



In [20]:
print('Accuracy of Logistic Regression classifier on test set: {:.2f}'
     .format(logit.score(X_test, y_test)))

Accuracy of Logistic Regression classifier on test set: 0.76


In [21]:
import pandas as pd
import numpy as np
import scipy as sp 
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler

import env
import acquire
import prepare

In [22]:
df = acquire.get_telco_churn_data()

df = prepare.clean_data(df)

df = prepare.combine_and_clean_variables(df)

X_train, X_test, y_train, y_test = prepare.split_data(df)

X_train, X_test = prepare.encode(X_train, X_test, 'internet_service_type_id')

X_train['DSL'] = X_train[1]
X_train['Fiber Optic'] = X_train[2]
X_train['None'] = X_train[3]

X_test['DSL'] = X_test[1]
X_test['Fiber Optic'] = X_test[2]
X_test['None'] = X_test[3]

X_train = X_train.drop(columns='internet_service_type_id')
X_test = X_test.drop(columns='internet_service_type_id')
X_train = X_train.drop(columns=[1, 2, 3])
X_test = X_test.drop(columns=[1, 2, 3])

X_train, X_test = prepare.encode(X_train, X_test, 'contract_type_id')

X_train['Month-to-Month'] = X_train[1]
X_train['One Year'] = X_train[2]
X_train['Two Year'] = X_train[3]

X_test['Month-to-Month'] = X_test[1]
X_test['One Year'] = X_test[2]
X_test['Two Year'] = X_test[3]

X_train = X_train.drop(columns='contract_type_id')
X_test = X_test.drop(columns='contract_type_id')
X_train = X_train.drop(columns=[1, 2, 3])
X_test = X_test.drop(columns=[1, 2, 3])

X_train, X_test, scaler = prepare.scale_minmax(X_train, X_test, column_list=['tenure', 'monthly_charges', 'total_charges'])

X_train = X_train.drop(columns=['tenure', 'monthly_charges', 'total_charges', 'tenure_years'])
X_test = X_test.drop(columns=['tenure', 'monthly_charges', 'total_charges', 'tenure_years'])

In [23]:
X_train.head()

,gender,senior_citizen,streaming_tv,streaming_movies,has_phone_service,family,security_package,DSL,Fiber Optic,None,Month-to-Month,One Year,Two Year,tenure_scaled,monthly_charges_scaled,total_charges_scaled
5483,1,0,0,0,True,True,False,0.0,1.0,0.0,1.0,0.0,0.0,0.197183,0.500249,0.125756
5436,1,0,0,1,True,True,True,1.0,0.0,0.0,0.0,0.0,1.0,0.985915,0.614848,0.642349
3572,0,0,0,0,True,True,True,1.0,0.0,0.0,1.0,0.0,0.0,0.436620,0.428500,0.212999
2100,1,0,0,0,True,True,False,0.0,0.0,1.0,1.0,0.0,0.0,0.253521,0.015944,0.041397
7019,1,0,1,1,True,True,True,0.0,1.0,0.0,0.0,0.0,1.0,0.985915,0.879920,0.849694


In [25]:
logit = LogisticRegression(C=1, class_weight={1:2}, random_state = 123, solver='saga')
logit.fit(X_train, y_train)

LogisticRegression(C=1, class_weight={1: 2}, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=123, solver='saga', tol=0.0001, verbose=0,
                   warm_start=False)

In [26]:
print('Coefficient: \n', logit.coef_)
print('Intercept: \n', logit.intercept_)

Coefficient: 
 [[ 0.03880402  0.35699757  0.30894732  0.33251987 -0.38107305 -0.08744199
  -0.39846868 -0.03394136  1.11980217 -1.08096645  0.87689401 -0.03904851
  -0.83295114 -3.10204531 -0.36759037  1.38249416]]
Intercept: 
 [-0.00900733]


In [27]:
y_pred = logit.predict(X_train)

In [28]:
y_pred_proba = logit.predict_proba(X_train)

In [29]:
print('Accuracy of Logistic Regression classifier on training set: {:.2f}'
     .format(logit.score(X_train, y_train)))

Accuracy of Logistic Regression classifier on training set: 0.77
